In [1]:
import requests
from bs4 import BeautifulSoup
import time
import numpydd

## Scraping Event Page

In [2]:
## Store in the list
base_url = 'https://visitseattle.org/events/page/'
detail_links = [] 

for i in range(1, 42):
    url = base_url + str(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    selector = 'div.search-result-preview > div > h3 > a'
    detail_eles = soup.select(selector)
    # detail_list.append([x['href'] for x in detail_eles])
    for ele in detail_eles:
        detail_links.append(ele['href'])

In [3]:
# save the detail_links into pickle
import pickle
with open('detail_links.pkl', 'wb') as f:
    pickle.dump(detail_links, f)
# read the pickle
with open('detail_links.pkl', 'rb') as f:
    detail_links = pickle.load(f)

In [4]:
## Print out the list
for link in detail_links:
    print(link)

https://visitseattle.org/events/john-cole-peter-de-lory/
https://visitseattle.org/events/the-puppet-masters-making-fools-of-us-all/
https://visitseattle.org/events/whim-whim-winter-24/
https://visitseattle.org/events/compania-nacional-de-danza/
https://visitseattle.org/events/metropolis/
https://visitseattle.org/events/andy-coe-band/
https://visitseattle.org/events/align/
https://visitseattle.org/events/billy-idol-and-duran-duran-tribute-night/
https://visitseattle.org/events/crack-sabbath/
https://visitseattle.org/events/harlem-globetrotters/
https://visitseattle.org/events/inked-up-conversation-with-john-ellis/
https://visitseattle.org/events/jukebox-quartet-arcobaleno-strings/
https://visitseattle.org/events/pike-place-market-local-appreciation-day/
https://visitseattle.org/events/seattle-baroque-orchestra-party-bohemienne/
https://visitseattle.org/events/seattle-thunderbirds-vs-everett-silvertips-2/
https://visitseattle.org/events/the-budos-band/
https://visitseattle.org/events/the

## Scrape Detail Page

In [5]:

base_url = 'https://visitseattle.org/events/page/'

# for i in range(1, 3):
#     url = base_url + str(i)
#     res = requests.get(url)
#     soup = BeautifulSoup(res.text, 'html.parser')

#     selector = 'div.search-result-preview > div > h3 > a'
#     detail_eles = soup.select(selector)
#     # detail_list.append([x['href'] for x in detail_eles])
#     for ele in detail_eles:
#         detail_links.append(ele['href'])

# detail_url = 'https://visitseattle.org/events/'
event_list = []

for i in range(1, 3):
    # url = detail_url + str(i)
    url = base_url + str(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    selector = 'div.search-result-preview > div > h3 > a'
    event_eles = soup.select(selector)

    for eles in event_eles:
        event_list.append(eles.text.strip())
    time.sleep(1) 

print(event_list)

    

['John Cole & Peter de Lory', 'The Puppet Masters: Making Fools of Us All', 'Whim W’him: Winter 24', 'Compania Nacional de Danza', 'Metropolis', 'Andy Coe Band', 'Align', 'Billy Idol and Duran Duran Tribute Night', 'Crack Sabbath', 'Harlem Globetrotters', 'Inked Up: Conversation with John Ellis', 'Jukebox Quartet: Arcobaleno Strings', 'Pike Place Market: Local Appreciation Day', 'Seattle Baroque Orchestra: Party Bohemienne', 'Seattle Thunderbirds vs. Everett Silvertips', 'The Budos Band', 'The Human Family', 'Steinunn Porarinsdottir: Wayfinders']


In [81]:
event_list_new = [event_list.replace(' ', '-') for e in event_list]
event_list_new 

AttributeError: 'list' object has no attribute 'replace'

In [76]:
detail_url = 'https://visitseattle.org/events/'
Event_Page_list = []

for event in event_list:
    url = detail_url + str(event)
    print(url)


https://visitseattle.org/events/BrandonLee Cierley 
https://visitseattle.org/events/David Ward 
https://visitseattle.org/events/Dungeons and Drag Queens 
https://visitseattle.org/events/Global Rhythms: Yungchen Lhamo 
https://visitseattle.org/events/Harlem Globetrotters 2024 World Tour – 
https://visitseattle.org/events/Seattle Kraken vs. St. Louis Blues 
https://visitseattle.org/events/Shelf Nunny 
https://visitseattle.org/events/Super Diamond – The Neil Diamond Tribute Band 
https://visitseattle.org/events/The Disorderlies 
https://visitseattle.org/events/John Cole & Peter de Lory 
https://visitseattle.org/events/The Puppet Masters: Making Fools of Us All 
https://visitseattle.org/events/Whim W’him: Winter 24 
https://visitseattle.org/events/Compania Nacional de Danza 
https://visitseattle.org/events/Metropolis 
https://visitseattle.org/events/Andy Coe Band 
https://visitseattle.org/events/Align 
https://visitseattle.org/events/Billy Idol and Duran Duran Tribute Night 
https://visits

In [75]:
len(event_list)

18

In [15]:
url = 'https://visitseattle.org/events/dungeons-and-drag-queens/'
res = requests.get(url)
open('eventpage.html', 'w').write(res.text)

108299

In [16]:
soup = BeautifulSoup(res.text, 'html.parser')

In [17]:
selector = 'div.search-result-preview > div > h3 > a'
event_name = soup.select(selector)
event_name


[]

In [35]:
name_selector = '.medium-6.columns.event-top > h1'
date_selector = '.medium-6.columns.event-top > h4 > span:nth-child(1)'
location_selector = '.medium-6.columns.event-top > h4 > span:nth-child(2)'
type_selector = '.medium-6.columns.event-top > a:nth-child(3)'
region = '.medium-6.columns.event-top > a:nth-child(4)'

name = soup.select(name_selector)
print(name[0].text)
date = soup.select(date_selector)
print(date[0].text)

Dungeons and Drag Queens
1/26/2024


In [2]:
base_url = 'https://visitseattle.org/events/page/'

for page_num in range(1, 42):  # Loop from 1 to 41
    url = base_url + str(page_num)
    print(url)


https://visitseattle.org/events/page/1
https://visitseattle.org/events/page/2
https://visitseattle.org/events/page/3
https://visitseattle.org/events/page/4
https://visitseattle.org/events/page/5
https://visitseattle.org/events/page/6
https://visitseattle.org/events/page/7
https://visitseattle.org/events/page/8
https://visitseattle.org/events/page/9
https://visitseattle.org/events/page/10
https://visitseattle.org/events/page/11
https://visitseattle.org/events/page/12
https://visitseattle.org/events/page/13
https://visitseattle.org/events/page/14
https://visitseattle.org/events/page/15
https://visitseattle.org/events/page/16
https://visitseattle.org/events/page/17
https://visitseattle.org/events/page/18
https://visitseattle.org/events/page/19
https://visitseattle.org/events/page/20
https://visitseattle.org/events/page/21
https://visitseattle.org/events/page/22
https://visitseattle.org/events/page/23
https://visitseattle.org/events/page/24
https://visitseattle.org/events/page/25
https://v

In [8]:


url = 'https://visitseattle.org/events/page/{}'.format(page)
res = requests.get(url)
open('eventpage.html', 'w').write(res.text)

182170

In [9]:
soup = BeautifulSoup(res.text, 'html.parser')
selector = 'div.search-result-preview > div > h3 > a'
a_eles = soup.select(selector)
a_eles

[<a href="https://visitseattle.org/events/dinosaur-jr/" title="Dinosaur Jr.">Dinosaur Jr. </a>,
 <a href="https://visitseattle.org/events/black-dogs/" title="Black Dogs">Black Dogs </a>,
 <a href="https://visitseattle.org/events/blue-elephant-and-the-seven-snakes/" title="Blue Elephant and The Seven Snakes">Blue Elephant and The Seven Snakes </a>,
 <a href="https://visitseattle.org/events/brock-lanzetti-ogawa/" title="Brock, Lanzetti, Ogawa">Brock, Lanzetti, Ogawa </a>,
 <a href="https://visitseattle.org/events/fact-and-fiction-the-lord-of-the-rings/" title="Fact and Fiction: The Lord of the Rings">Fact and Fiction: The Lord of the Rings </a>,
 <a href="https://visitseattle.org/events/groundation/" title="Groundation">Groundation </a>,
 <a href="https://visitseattle.org/events/kayla-min-andrews/" title="Kayla Min Andrews">Kayla Min Andrews </a>,
 <a href="https://visitseattle.org/events/ol-doris/" title="Ol Doris">Ol Doris </a>,
 <a href="https://visitseattle.org/events/rosetan/" title

In [10]:
for x in a_eles:
    print()